# Validating QuASAr Simulation Correctness

This notebook exercises the high-level QuASAr API to verify that the latest subsystem descriptor (SSD) structure still produces physically correct simulation outcomes.  Each circuit is simulated via the `SimulationEngine`, compared against an explicit statevector reference, and the resulting fidelity and amplitude deviations are reported.

In [1]:
import sys
import pathlib

# Ensure the repository root is on sys.path so `import quasar` succeeds during execution.
ROOT = pathlib.Path.cwd().resolve()
if not (ROOT / 'quasar').exists():
    for candidate in ROOT.parents:
        if (candidate / 'quasar').exists():
            ROOT = candidate
            break
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))
print('Using repository root:', ROOT)

Using repository root: /workspace/QuASAr


## Imports and Helpers

The helper routine below evaluates a circuit with QuASAr's Aer-backed statevector simulator so that we can compare the engine's output against a dense reference state.

In [2]:
import math
import numpy as np

from quasar import Circuit, Gate, SimulationEngine, Backend, StatevectorBackend

np.set_printoptions(precision=6, suppress=True)

In [3]:
def reference_state(circuit: Circuit) -> np.ndarray:
    """Return a dense statevector for `circuit` using Aer."""
    backend = StatevectorBackend()
    backend.load(circuit.num_qubits)
    for gate in circuit.gates:
        backend.apply_gate(gate.gate, gate.qubits, gate.params)
    return np.asarray(backend.statevector(), dtype=complex)


## Benchmark Circuits

We exercise a set of representative circuits that touch rotations, entanglement, controlled phase operations, and multi-qubit interactions.

In [4]:
circuits = {
    "single_qubit_rotations": Circuit(
        [
            Gate("H", [0]),
            Gate("RZ", [0], {"param0": float(np.pi / 3)}),
            Gate("RX", [0], {"param0": float(-np.pi / 5)}),
            Gate("RY", [0], {"param0": float(np.pi / 7)}),
        ],
        use_classical_simplification=False,
    ),
    "bell_pair": Circuit(
        [
            Gate("H", [0]),
            Gate("CX", [0, 1]),
            Gate("RZ", [1], {"param0": float(np.pi / 4)}),
            Gate("CX", [0, 1]),
        ],
        use_classical_simplification=False,
    ),
    "three_qubit_mixer": Circuit(
        [
            Gate("H", [0]),
            Gate("CX", [0, 1]),
            Gate("RY", [2], {"param0": float(np.pi / 5)}),
            Gate("CZ", [1, 2]),
            Gate("T", [0]),
            Gate("SX", [1]),
            Gate("RZZ", [0, 2], {"param0": float(-np.pi / 7)}),
            Gate("CP", [0, 2], {"k": 3}),
        ],
        use_classical_simplification=False,
    ),
}
circuits


{'single_qubit_rotations': <quasar.circuit.Circuit at 0x7f349ea79ac0>,
 'bell_pair': <quasar.circuit.Circuit at 0x7f349ea7aed0>,
 'three_qubit_mixer': <quasar.circuit.Circuit at 0x7f349ea7bb60>}

## Simulation and Validation

Each circuit is simulated via the `SimulationEngine`.  We request the dense statevector backend so that the final SSD contains an explicit amplitude vector.  The notebook reports the resulting fidelity against the reference, the maximum per-amplitude deviation, and the backends used for each partition in the SSD.

In [5]:
engine = SimulationEngine()
summary_rows = []
detailed_results = {}

for name, circuit in circuits.items():
    ref = reference_state(circuit)
    result = engine.simulate(circuit, backend=Backend.STATEVECTOR, reference_state=ref)
    partition_backends = [part.backend.name for part in result.ssd.partitions]
    extracted = result.ssd.extract_state(0) if result.ssd.partitions else None
    final_state = np.asarray(extracted, dtype=complex) if extracted is not None else None
    max_error = float(np.max(np.abs(final_state - ref))) if final_state is not None else float('nan')
    summary_rows.append({
        "Circuit": name,
        "Qubits": circuit.num_qubits,
        "Partitions": len(result.ssd.partitions),
        "Backends": partition_backends,
        "Fidelity": result.fidelity,
        "Max amplitude error": max_error,
    })
    detailed_results[name] = {
        "circuit": circuit,
        "result": result,
        "reference": ref,
        "final_state": final_state,
    }

    print("{}: fidelity={:.6f}, max_error={:.3e}, backends={}".format(name, result.fidelity, max_error, partition_backends))

try:
    import pandas as pd
except Exception:
    pd = None

if pd is not None:
    display(pd.DataFrame(summary_rows))
else:
    summary_rows


single_qubit_rotations: fidelity=1.000000, max_error=0.000e+00, backends=['STATEVECTOR']
bell_pair: fidelity=1.000000, max_error=0.000e+00, backends=['STATEVECTOR']
three_qubit_mixer: fidelity=1.000000, max_error=0.000e+00, backends=['STATEVECTOR']


,Circuit,Qubits,Partitions,Backends,Fidelity,Max amplitude error
0,single_qubit_rotations,1,1,[STATEVECTOR],1.0,0.0
1,bell_pair,2,1,[STATEVECTOR],1.0,0.0
2,three_qubit_mixer,3,1,[STATEVECTOR],1.0,0.0


## Inspecting a Final State

The following cell compares QuASAr's terminal amplitudes for the three-qubit mixer against the dense reference vector.

In [6]:
target = detailed_results["three_qubit_mixer"]
ref = target["reference"]
state = target["final_state"]
print("Reference amplitudes:")
print(ref)
print("QuASAr amplitudes:")
print(state)
print("Max absolute deviation:", np.max(np.abs(ref - state)))


Reference amplitudes:
[ 0.252996+0.402641j  0.130826+0.082203j  0.402641-0.252996j
  0.082203-0.130826j  0.463606-0.105815j -0.082203-0.130826j
  0.105815+0.463606j  0.130826-0.082203j]
QuASAr amplitudes:
[ 0.252996+0.402641j  0.130826+0.082203j  0.402641-0.252996j
  0.082203-0.130826j  0.463606-0.105815j -0.082203-0.130826j
  0.105815+0.463606j  0.130826-0.082203j]
Max absolute deviation: 0.0
